In [1]:
import json
with open(".\\data\\file_references.json") as fd:
    repos = json.load(fd)

def get_element(tag):
    elements = []
    for parent in tag.parents:
        if parent.name is None:
            break
        siblings = [sib for sib in parent.children if sib.name == tag.name]
        if len(siblings) > 1:
            elements.append(f'{tag.name}[{siblings.index(tag) + 1}]')
        else:
            elements.append(tag.name)
        tag = parent
    elements.reverse()
    return '/' + '/'.join(elements)



In [32]:
import requests
from bs4 import BeautifulSoup
import json

with open("./data/file_references.json", "r") as fd:
    repos = json.load(fd)

for idx in range(181,182):
    url = repos[idx]
    style_assoc = {}
    response = requests.get("https://raw.githubusercontent.com/jernesto91/sabores2024-catalogo/master/src/app/components/social-networks/social-networks.component.html?token=GHSAT0AAAAAACR7O5OJTOZ5WNOMFM6LFPN2ZSU6AHA")
        # Check if the request was successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        tags_with_specific_property = soup.find_all(attrs={"style": True})
        if tags_with_specific_property:
            print(f"\n[{idx}] : {url}")
        for tag in tags_with_specific_property:
            xpath = get_element(tag)
            print(f"{xpath} : {tag['style'].strip()}")
            for s in tag['style'].strip().split(";"):
                style_assoc.setdefault(get_element(tag),[]).append(s)
        idx = idx + 1
    else:
        print(f"Failed to fetch the URL. Status code: {response.status_code}")





[181] : https://raw.githubusercontent.com/swimlane/ngx-charts/ff89cfe08ada08ab898083be76cd0a6cc1adfdb2/src/app/app.component.html
/ion-grid : max-width: 500px; min-width: fit-content;
/ion-grid/ion-row/ng-container/ion-col : min-width: fit-content;
/ion-grid/ion-row/ng-container/ion-col/a/img : min-height: 36px; min-width: 36px;


Verify that the set is correct

In [33]:
import collections
od = collections.OrderedDict(sorted(style_assoc.items()))
for k , v in od.items():
    print(f"{k} : {v}")





/ion-grid : ['max-width: 500px', ' min-width: fit-content', '']
/ion-grid/ion-row/ng-container/ion-col : ['min-width: fit-content', '']
/ion-grid/ion-row/ng-container/ion-col/a/img : ['min-height: 36px', ' min-width: 36px', '']


In [36]:
# Invert the dictionary.
from collections import defaultdict
from typing import Dict, List, DefaultDict

inverted_dictionary: DefaultDict[str, List[str]] = defaultdict(list)

for tag_position, styles in od.items():
    for style in styles:
        if len(style.strip()) >= 1:
            inverted_dictionary[style.strip()].append(tag_position)

sorted_inverted_dict: Dict[str, List[str]] = dict(
    sorted(inverted_dictionary.items(), key=lambda item: len(item[1]), reverse=False)
)

grouped_1_len: DefaultDict[str, List[str]] = defaultdict(list)

for k, v in sorted_inverted_dict.items():
    if len(v) <= 1:
        print(f"{k} : {v}")

def generate_css_classes(mapping: Dict[str, List[str]]) -> Dict[str, List[str]]:
    # Group elements by their CSS properties
    grouped_properties: DefaultDict[str, List[str]] = defaultdict(list)
    for css_property, elements in mapping.items():
        for element in elements:
            grouped_properties[element].append(css_property)
    
    # Generate CSS classes
    css_classes: Dict[str, List[str]] = {}
    for i, (element, properties) in enumerate(grouped_properties.items()):
        class_name = f'class-{i+1}'
        css_classes[class_name] = properties

    return css_classes

def merge_identical_classes(class_mappings: Dict[str, List[str]]) -> Dict[str, List[str]]:
    property_to_class: DefaultDict[tuple, List[str]] = defaultdict(list)

    # Map property sets to class names
    for class_name, properties in class_mappings.items():
        property_key = tuple(sorted(properties))
        property_to_class[property_key].append(class_name)

    # Create a new dictionary to store the merged classes
    merged_classes: Dict[str, List[str]] = {}
    for properties, class_names in property_to_class.items():
        if len(class_names) > 1:
            new_class_name = class_names[0]
            merged_classes[new_class_name] = list(properties)
        else:
            merged_classes[class_names[0]] = list(properties)

    return merged_classes

css_classes = generate_css_classes(sorted_inverted_dict)
css_classes = merge_identical_classes(css_classes)

for k, v in css_classes.items():
    print(f"{k} : {v}")

max-width: 500px : ['/ion-grid']
min-height: 36px : ['/ion-grid/ion-row/ng-container/ion-col/a/img']
min-width: 36px : ['/ion-grid/ion-row/ng-container/ion-col/a/img']
class-1 : ['max-width: 500px', 'min-width: fit-content']
class-2 : ['min-height: 36px', 'min-width: 36px']
class-3 : ['min-width: fit-content']
